In [6]:
"""Plague Classification Model with ArcGIS Data - Improved Version"""

import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing import image_dataset_from_directory
from PIL import Image
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report
import logging
import time

# Parámetros de la red
EPOCHS = 50
IMAGE_SIZE = (224, 224)  # Usamos 224x224 para MobileNetV2
INPUT_SHAPE = (224, 224, 3)
SEED = 123
BATCH_SIZE = 32
BUFFER_SIZE = 350
LEARNING_RATE = 1e-4

# # Montar Google Drive
# from google.colab import drive
# drive.mount('/content/drive')


# Directorio de las imágenes
images_dir = '../arcgis-survey-images-new-last/arcgis-survey-images-new-last'

# Número máximo de imágenes por clase
MAX_IMAGES_PER_CLASS = 500

# Función para limitar las imágenes a un máximo por clase
def filter_max_images_per_class(ds, max_images_per_class, class_names):
    class_counts = {class_name: 0 for class_name in class_names}
    
    def filter_fn(image, label):
        class_name = class_names[label.numpy()]
        if class_counts[class_name] < max_images_per_class:
            class_counts[class_name] += 1
            return True
        else:
            return False

    return ds.filter(lambda image, label: tf.py_function(
        filter_fn, [image, label], tf.bool))

# División de los datos en entrenamiento, validación y prueba
train_ds = image_dataset_from_directory(
    images_dir,
    labels="inferred",
    label_mode="int",
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    shuffle=True
)

val_ds = image_dataset_from_directory(
    images_dir,
    labels="inferred",
    label_mode="int",
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    validation_split=0.2,
    subset="validation",
    seed=SEED
)

# Obtener los nombres de las clases
class_names = train_ds.class_names
num_classes = len(class_names)

# Limitar las imágenes a 500 por clase en el dataset de entrenamiento
train_ds = filter_max_images_per_class(train_ds, MAX_IMAGES_PER_CLASS, class_names)

# Crear un conjunto de prueba a partir del conjunto de validación
val_batches = tf.data.experimental.cardinality(val_ds)
test_ds = val_ds.take(val_batches // 2)
val_ds = val_ds.skip(val_batches // 2)

# Aplicar Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal_and_vertical'),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomBrightness(0.2),
])

# Función para aplicar Data Augmentation y preprocesamiento
def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image = data_augmentation(image)
    return image, label

train_ds = train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

# Optimizar rendimiento de los datasets
train_ds = train_ds.cache().shuffle(BUFFER_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)


# Cargar el modelo base (MobileNetV2)
base_model = MobileNetV2(input_shape=INPUT_SHAPE,
                         include_top=False,
                         weights='imagenet')

# Congelar las capas iniciales del modelo base
base_model.trainable = True
fine_tune_at = 100  # Descongelar desde la capa 100 en adelante

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Construir el modelo completo
inputs = tf.keras.Input(shape=INPUT_SHAPE)
x = data_augmentation(inputs)
x = layers.Rescaling(1./127.5, offset=-1)(x)  # Escalar entre [-1, 1] para MobileNetV2
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
model = models.Model(inputs, outputs)

# Compilar el modelo
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks para Early Stopping y Model Checkpointing
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint_filepath = '/tmp/checkpoint.weights.h5'
model_checkpoint = ModelCheckpoint(filepath=checkpoint_filepath,
                                   save_weights_only=True,
                                   monitor='val_accuracy',
                                   mode='max',
                                   save_best_only=True)


# Entrenamiento del modelo
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[early_stopping, model_checkpoint]
)

# Cargar los mejores pesos del modelo
model.load_weights(checkpoint_filepath)

# Gráfica de la pérdida y precisión
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Entrenamiento')
plt.plot(epochs_range, val_acc, label='Validación')
plt.legend(loc='lower right')
plt.title('Precisión')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Entrenamiento')
plt.plot(epochs_range, val_loss, label='Validación')
plt.legend(loc='upper right')
plt.title('Pérdida')
plt.show()

# Evaluación en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"\nResultados de evaluación en test set:")
print(f"Pérdida en test: {test_loss:.4f}")
print(f"Precisión en test: {test_accuracy:.4f}")

# Matriz de confusión y reporte de clasificación
y_pred = []
y_true = []

for images, labels in test_ds:
    preds = model.predict(images)
    y_pred.extend(np.argmax(preds, axis=1))
    y_true.extend(labels.numpy())

conf_matrix = tf.math.confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, xticklabels=class_names, yticklabels=class_names, annot=True, fmt='g')
plt.xlabel('Predicción')
plt.ylabel('Etiqueta Verdadera')
plt.title('Matriz de Confusión')
plt.show()

print("Reporte de clasificación:")
print(classification_report(y_true, y_pred, target_names=class_names))


Found 4308 files belonging to 5 classes.
Using 3447 files for training.
Found 4308 files belonging to 5 classes.
Using 861 files for validation.
Epoch 1/50
     77/Unknown 76s 756ms/step - accuracy: 0.3823 - loss: 1.5664

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


ValueError: Input 0 of layer "functional_9" is incompatible with the layer: expected shape=(None, 224, 224, 3), found shape=(224, 224, 3)

In [ ]:
# Guardar el modelo en formato .keras
model.save('MobileNetV2-79%.keras')